In [1]:
import warnings
warnings.filterwarnings('ignore')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = "2"

import time
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold, StratifiedKFold
import gc

import torch as t
from torch.utils.data import DataLoader
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [2]:
%%time
label_1 = pd.read_csv('./data/train_preliminary/user.csv')
label_2 = pd.read_csv('./data/train_semi_final/user.csv')
label = pd.concat([label_1, label_2], axis=0).reset_index(drop=True)
mats_train = []
mats_test = []
for col in tqdm(['creative_id', 'ad_id', 'advertiser_id', 'product_id', 'industry']):    
    mats_train.append(np.load('./inputs_new/{}_inputs_train.npy'.format(col)))
    mats_test.append(np.load('./inputs_new/{}_inputs_test.npy'.format(col)))

100%|██████████| 5/5 [00:06<00:00,  1.34s/it]

CPU times: user 400 ms, sys: 6.92 s, total: 7.32 s
Wall time: 7.32 s


In [3]:
import logging

def get_logger(filename, verbosity=1, name=None):
    level_dict = {0: logging.DEBUG, 1: logging.INFO, 2: logging.WARNING}
    formatter = logging.Formatter(
        "[%(asctime)s][%(filename)s][line:%(lineno)d][%(levelname)s] %(message)s"
    )
    logger = logging.getLogger(name)
    logger.setLevel(level_dict[verbosity])

    fh = logging.FileHandler(filename, "w")
    fh.setFormatter(formatter)
    logger.addHandler(fh)

    sh = logging.StreamHandler()
    sh.setFormatter(formatter)
    logger.addHandler(sh)

    return logger

In [4]:
class LSTM(nn.Module):
    def __init__(self):
        super(LSTM, self).__init__() 
        emb_outputs = []
        
        cols = ['creative_id', 'ad_id', 'advertiser_id', 'product_id', 'industry']
        n_in = len(cols)
        for i in range(n_in):
            We = np.load('./w2v_256_120/{}_embedding_weight.npy'.format(cols[i]))
            We = np.vstack([We, np.zeros(256)])
            embed = nn.Embedding(num_embeddings=We.shape[0],embedding_dim=We.shape[1],padding_idx=len(We)-1, _weight=t.FloatTensor(We))
            for p in embed.parameters(): 
                p.requires_grad=False
            emb_outputs.append(embed)
            
        for i in range(n_in):
            We = np.load('./w2v_128_60/{}_embedding_weight.npy'.format(cols[i]))
            We = np.vstack([We, np.zeros(128)])
            embed = nn.Embedding(num_embeddings=We.shape[0],embedding_dim=We.shape[1],padding_idx=len(We)-1, _weight=t.FloatTensor(We))
            for p in embed.parameters(): 
                p.requires_grad=False
            emb_outputs.append(embed)
            del We
            gc.collect()

        self.encoders = nn.ModuleList(emb_outputs)
        self.emb_drop = nn.Dropout(p=0.2)
        self.lstm = nn.LSTM(input_size=(256+128)*5, hidden_size=384, num_layers=2, bias=True, batch_first=True, dropout = 0.2, bidirectional = True)
        self.max_pool = nn.MaxPool1d(kernel_size=2, stride=2)
        self.fc = nn.Sequential(nn.Linear(384, n_cls))
        self.fc_drop = nn.Dropout(p=0.2)
        

    def forward(self, xs):
        inp = [self.encoders[i](x) for i, x in enumerate(xs)] + [self.encoders[i + 5](x) for i, x in enumerate(xs)]
        x = t.cat(inp, 2)
        x = self.emb_drop(x)
        x = self.lstm(x)[0]
        x = self.max_pool(x)
        x = t.max(x, dim=1)[0]
        x = self.fc_drop(x)
        logits = self.fc(x)
        return logits

In [5]:
def clip_gradient(optimizer, grad_clip):
    for group in optimizer.param_groups:
        #print(group['params'])
        for param in group['params']:
            param.grad.data.clamp_(-grad_clip, grad_clip)

In [6]:
class Dataset(t.utils.data.Dataset):
    def __init__(self, xs, y, shuffle=False):
        self.xs = xs
        self.y = y
        self.size = len(xs[0])
        self.shuffle = shuffle
         
    def __len__(self):
        return self.size
    
    def __getitem__(self, idx):
        xs = [x[idx].astype(np.int64) for x in self.xs]
        y = self.y[idx]
        if self.shuffle and np.random.rand() < 0.8:
            state = np.random.get_state()
            for x in xs:
                np.random.set_state(state)
                np.random.shuffle(x)     
        return xs, y

In [7]:
def train(model, loader, optimizer, criterion):
    t0 = time.time()
    y_true = []
    y_pred = []
    model.train() 
    for i, (xs, y) in enumerate(loader):
        optimizer.zero_grad()
        xs = [t.LongTensor(x).cuda() for x in xs]
        yp = model(xs)
        loss = criterion(yp, t.LongTensor(y).cuda())
        loss.backward()
        clip_gradient(optimizer, 0.1)
        optimizer.step()
        yp = t.softmax(yp, 1)
        yp = np.argmax(yp.detach().cpu().numpy(), axis=1)
        y_pred.append(yp)
        y_true.append(y)
        print('process: [%d/%d]' % (i + 1, len(loader)), end='\r')
    y_true = np.hstack(y_true)
    y_pred = np.hstack(y_pred)
    score = accuracy_score(y_true, y_pred)
    print('process: [%d/%d], score: %f, dtime: %ds' % (i + 1, len(loader), score, time.time()-t0), end='\n')
    return None

def val(model, loader):
    t0 = time.time()
    y_true = []
    y_pred = []
    model.eval()
    with t.no_grad():
        for i, (xs, y) in enumerate(loader):
            xs = [t.LongTensor(x).cuda() for x in xs]
            yp = model(xs)
            yp = t.softmax(yp, 1)
            yp = np.argmax(yp.cpu().numpy(), axis=1)
            y_pred.append(yp)
            y_true.append(y)
            print('process: [%d/%d]' % (i + 1, len(loader)), end='\r')
    model.train() 
    y_true = np.hstack(y_true)
    y_pred = np.hstack(y_pred)
    score = accuracy_score(y_true, y_pred)
    print('process: [%d/%d], score: %f, dtime: %ds' % (i + 1, len(loader), score, time.time()-t0), end='\n')
    return y_pred, score

def test(model, loader, y_test):
    t0 = time.time()
    y_true = []
    y_pred = []
    model.eval()
    with t.no_grad():
        for i, (xs,y) in enumerate(loader):
            xs = [t.LongTensor(x).cuda() for x in xs]
            yp = model(xs)
            yp = t.softmax(yp, 1)
            y_test[i*batch_size:(i+1)*batch_size] = yp.cpu().numpy()
            print('process: [%d/%d]' % (i + 1, len(loader)), end='\r')
    model.train() 
    print('process: [%d/%d], dtime: %ds' % (i + 1, len(loader), time.time()-t0), end='\n')
    return y_test

In [ ]:
tmp_data = pd.read_csv('./data/train_preliminary/user.csv')
y = label['age'].values-1
n_cls = 10
batch_size = 1024
y_test = np.zeros([mats_test[0].shape[0], n_cls])
loader_te = DataLoader(Dataset(mats_test, np.zeros(mats_test[0].shape[0])), batch_size=batch_size, shuffle=False, num_workers=4)
kfold = StratifiedKFold(n_splits=5,random_state=2020)
best_scores = []
logger = get_logger('./torch1.log')
for i, (idx_trn, idx_val) in (enumerate(kfold.split(tmp_data, tmp_data['age']))):  
    logger.info('fold {} start training!'.format(i))
    all_idx = np.arange(len(mats_train[0]))
    subtraction = list((set(all_idx).difference(set(idx_val))))
    x_test = np.zeros([mats_test[0].shape[0], n_cls])
    x_trn = [x[subtraction] for x in mats_train]
    x_val = [x[idx_val] for x in mats_train]
    print(len(x_trn[0]), len(x_val[0]))
    y_trn, y_val = y[subtraction], y[idx_val]
#     x_trn = [mat[idx_trn] for mat in mats_train]
#     x_val = [mat[idx_val] for mat in mats_train]
#     y_trn, y_val = y[idx_trn], y[idx_val]
    model = LSTM().cuda()
    optimizer = t.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr = 0.001)
    scheduler = t.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)
    criterion = nn.CrossEntropyLoss()
    loader_trn = DataLoader(Dataset(x_trn, y_trn, shuffle=True), batch_size=batch_size, shuffle=True, num_workers=4)
    loader_val = DataLoader(Dataset(x_val, y_val), batch_size=batch_size, shuffle=False, num_workers=4)
    best_score = 0.0
    for i in range(20):
        if i > 10:
            scheduler.step()
        train(model, loader_trn, optimizer, criterion)
        result, score = val(model, loader_val)
        logger.info('Epoch:[{}/{}]\t score={:.5f}\t'.format(i , 20, score))
        if score > best_score:
            x_test = test(model, loader_te, x_test)
            best_score = score
            print('best score: %.5f' % score)
    best_scores.append(best_score)
    y_test += x_test
print('best scores:', best_scores)

[2020-07-20 02:21:16,484][<ipython-input-8-63381bc11791>][line:11][INFO] fold 0 start training!


2820000 180000
process: [2754/2754], score: 0.460372, dtime: 2410s


[2020-07-20 03:02:51,277][<ipython-input-8-63381bc11791>][line:34][INFO] Epoch:[0/20]	 score=0.48811	


process: [176/176], score: 0.488106, dtime: 52s
process: [977/977], dtime: 284s
best score: 0.48811
process: [2754/2754], score: 0.486878, dtime: 2410s


[2020-07-20 03:48:38,116][<ipython-input-8-63381bc11791>][line:34][INFO] Epoch:[1/20]	 score=0.49632	


...

process: [176/176], score: 0.511472, dtime: 52s
process: [2754/2754], score: 0.523329, dtime: 2412s


[2020-07-20 11:56:38,499][<ipython-input-8-63381bc11791>][line:34][INFO] Epoch:[12/20]	 score=0.51272	


process: [176/176], score: 0.512722, dtime: 52s
process: [977/977], dtime: 284s
best score: 0.51272
process: [2754/2754], score: 0.525711, dtime: 2413s


[2020-07-20 12:42:29,171][<ipython-input-8-63381bc11791>][line:34][INFO] Epoch:[13/20]	 score=0.51402	


process: [176/176], score: 0.514017, dtime: 52s
process: [977/977], dtime: 284s
best score: 0.51402


In [10]:
np.save('./age_res/torch_lstm6_test_pred.npy', y_test/3)

In [9]:
np.sum(y_test[0])

3.000000044241041